# OMERO Micro-SAM Workflow




## 1. Setup and Installation

**Prerequisites:**
```bash
# Activate micro-sam environment
conda activate micro-sam

# Install the package
pip install -e .
```

**Note**: The micro-sam dependency must be installed separately:
```bash
conda install -c conda-forge micro-sam
```

In [1]:
# Import the streamlined package
import omero_annotate_ai
from omero_annotate_ai import (
    create_omero_connection_widget,
    create_workflow_widget,
    create_pipeline
)

# System imports
import os
from pathlib import Path

print(f"📦 omero-annotate-ai version: {omero_annotate_ai.__version__}")
print(f"🔗 Available widgets: Connection, Workflow")
print(f"✨ Streamlined 2-widget workflow ready!")

# Check dependencies
try:
    import ezomero
    print(f"🔗 OMERO functionality: ✅ Available")
except ImportError:
    print(f"🔗 OMERO functionality: ❌ Install with: pip install -e .[omero]")

try:
    import keyring
    print(f"🔑 Keyring support: ✅ Available")
except ImportError:
    print(f"🔑 Keyring support: ⚠️ Not available (manual password entry only)")

📦 omero-annotate-ai version: 0.1.0
🔗 Available widgets: Connection, Workflow
✨ Streamlined 2-widget workflow ready!
🔗 OMERO functionality: ✅ Available
🔑 Keyring support: ✅ Available


## 2. OMERO Connection

Use the secure connection widget to connect to your OMERO server. This widget provides:
- Auto-loading from `.env` and `.ezomero` files
- Secure password storage in OS keychain
- Connection testing and validation
- Password expiration options

In [ ]:
# Create and display the OMERO connection widget
print("🔌 OMERO Connection Setup")
print("Use the widget below to connect to your OMERO server:")
print("  • Fill in server details (host, username, password)")
print("  • Test connection before proceeding")
print("  • Choose password storage duration if desired")
print("  • Click 'Save & Connect' to establish connection")
print()

conn_widget = create_omero_connection_widget()
conn_widget.display()

print("\n📝 Next Step: Run the cell below after connecting to OMERO")

🔌 OMERO Connection Setup
Use the widget below to connect to your OMERO server:
  • Fill in server details (host, username, password)
  • Test connection before proceeding
  • Choose password storage duration if desired
  • Click 'Save & Connect' to establish connection

📄 Loaded configuration from connection history: root@localhost
🔐 Password loaded from keychain (no expiration)



📝 Next Step: Run the cell below after connecting to OMERO


In [3]:
# Get the OMERO connection
conn = conn_widget.get_connection()

if conn is None:
    raise ConnectionError("❌ No OMERO connection established. Please use the widget above to connect.")

print("✅ OMERO connection established!")
print(f"👤 User: {conn.getUser().getName()}")
print(f"🏢 Group: {conn.getGroupFromContext().getName()}")
print(f"🔐 Secure: {conn.isSecure()}")
print(f"🔗 Connection ready for workflow setup")

✅ OMERO connection established!
👤 User: root
🏢 Group: system
🔐 Secure: True
🔗 Connection ready for workflow setup


## 3. Workflow Configuration

The workflow widget provides a sequential, step-by-step process for configuring your annotation workflow:

### Workflow Steps:
1. **Select Working Directory** - Choose where to save annotations
2. **Choose Container** - Select OMERO container (dataset, project, etc.)
3. **Check Existing Tables** - Scan for existing annotation tables
4. **Configure Parameters** - Set micro-SAM and processing parameters
5. **Save Configuration** - Review and save your settings

### Features:
- Sequential workflow with progress tracking
- Automatic table management and naming
- Complete micro-SAM configuration
- Configuration preview and export

In [4]:
# Create and display the workflow widget
print("🔬 OMERO Annotation Workflow Setup")
print("Follow the sequential workflow below:")
print("  1. Select working directory")
print("  2. Choose OMERO container")
print("  3. Check existing annotation tables")
print("  4. Configure micro-SAM parameters")
print("  5. Save configuration")
print()

workflow_widget = create_workflow_widget(connection=conn)
workflow_widget.display()

print("\n📝 Next Step: Complete the workflow above, then run the cell below")

🔬 OMERO Annotation Workflow Setup
Follow the sequential workflow below:
  1. Select working directory
  2. Choose OMERO container
  3. Check existing annotation tables
  4. Configure micro-SAM parameters
  5. Save configuration




📝 Next Step: Complete the workflow above, then run the cell below


## 4. Pipeline Execution

Once you've completed the workflow configuration, we can run the complete annotation pipeline. This will:
1. Create or resume from annotation tracking table
2. Process images using micro-SAM
3. Launch napari for interactive annotation
4. Upload results to OMERO (or save locally in read-only mode)

In [ ]:
# Get configuration from workflow widget
config = workflow_widget.get_config()

# Validate configuration
try:
    config.validate()
    print("✅ Configuration is valid!")
except ValueError as e:
    print(f"❌ Configuration validation failed: {e}")
    raise

# Display configuration summary
print("\n📋 Configuration Summary:")
print(f"   📦 Container: {config.omero.container_type} (ID: {config.omero.container_id})")
print(f"   🎯 Training Set: {config.training.trainingset_name}")
print(f"   🔬 Model: {config.micro_sam.model_type}")
print(f"   📺 Channel: {config.omero.channel}")
print(f"   📁 Output: {config.batch_processing.output_folder}")
print(f"   🔄 Resume from Table: {config.workflow.resume_from_table}")
print(f"   📖 Read-only Mode: {config.workflow.read_only_mode}")

if config.patches.use_patches:
    print(f"   🧩 Patches: {config.patches.patches_per_image} per image ({config.patches.patch_size[0]}×{config.patches.patch_size[1]})")

if config.micro_sam.three_d:
    print(f"   🧊 3D processing: Enabled")

print(f"\n📊 Processing scope: {'All images' if config.training.segment_all else f'{config.training.train_n} training + {config.training.validate_n} validation'}")

In [ ]:
# Create pipeline and preview what will be processed
pipeline = create_pipeline(config, conn)

# Get container details
container_type = config.omero.container_type
container_id = config.omero.container_id

print(f"🔍 Validating {container_type} with ID {container_id}...")

# Validate container exists
container = conn.getObject(container_type.capitalize(), container_id)
if container is None:
    raise ValueError(f"{container_type.capitalize()} with ID {container_id} not found")

print(f"✅ Found {container_type}: {container.getName()}")
if container.getDescription():
    print(f"📝 Description: {container.getDescription()}")

# Get list of images that will be processed
try:
    images_list = pipeline.get_images_from_container()
    print(f"\n📊 Found {len(images_list)} images to process")
    
    # Show sample images
    print("\n🖼️ Sample images:")
    for i, img in enumerate(images_list[:5]):
        if hasattr(img, 'getName'):
            print(f"   {i+1}. {img.getName()} (ID: {img.getId()})")
        else:
            img_obj = conn.getObject("Image", img)
            if img_obj:
                print(f"   {i+1}. {img_obj.getName()} (ID: {img})")
    
    if len(images_list) > 5:
        print(f"   ... and {len(images_list) - 5} more images")
        
except Exception as e:
    print(f"❌ Error getting images from container: {e}")
    raise

print(f"\n✅ Ready to process {len(images_list)} images!")

In [ ]:
# Run the complete annotation pipeline
print("🚀 Starting annotation pipeline...")
print(f"   📊 Processing {len(images_list)} images")
print(f"   🔬 Using micro-SAM model: {config.micro_sam.model_type}")

if config.batch_processing.batch_size == 0:
    print(f"   ⚡ Processing: All images in one batch")
else:
    print(f"   📦 Processing: Batches of {config.batch_processing.batch_size} images")

print(f"   🎨 Napari will open for interactive annotation")
print(f"   📝 Close napari windows when annotation is complete")
print()

try:
    # Run the complete workflow - this is the key call!
    table_id, processed_images = pipeline.run_full_workflow()
    
    print(f"\n🎉 Annotation pipeline completed successfully!")
    print(f"📊 Processed {len(processed_images)} images")
    print(f"📋 Tracking table ID: {table_id}")
    
    if config.workflow.read_only_mode:
        print(f"💾 Annotations saved locally to: {config.batch_processing.output_folder}")
    else:
        print(f"☁️ Annotations uploaded to OMERO")
        
except Exception as e:
    print(f"❌ Error during annotation pipeline: {e}")
    import traceback
    traceback.print_exc()
    raise

## 5. Results and Export

Review the results and export your configuration for future use.

In [ ]:
# Display results summary
print("📊 Pipeline Results Summary:")
print(f"   🎯 Training Set: {config.training.trainingset_name}")
print(f"   📋 Tracking Table ID: {table_id}")
print(f"   📊 Images Processed: {len(processed_images)}")
print(f"   📦 Container: {config.omero.container_type} (ID: {config.omero.container_id})")
print(f"   🔬 Model Used: {config.micro_sam.model_type}")
print(f"   📁 Output Location: {config.batch_processing.output_folder}")

# Show processed images
if processed_images:
    print(f"\n🖼️ Processed Images:")
    for i, img_id in enumerate(processed_images[:10]):
        img_obj = conn.getObject("Image", img_id)
        if img_obj:
            print(f"   {i+1}. {img_obj.getName()} (ID: {img_id})")
    
    if len(processed_images) > 10:
        print(f"   ... and {len(processed_images) - 10} more images")

print(f"\n✅ Pipeline completed successfully!")

In [ ]:
# Export configuration for future use
config_filename = f"annotation_config_{config.training.trainingset_name}.yaml"
config_path = Path(config.batch_processing.output_folder) / config_filename

try:
    config.save_yaml(config_path)
    print(f"💾 Configuration saved to: {config_path}")
except Exception as e:
    print(f"⚠️ Could not save to output folder, saving to current directory")
    config.save_yaml(config_filename)
    print(f"💾 Configuration saved to: {config_filename}")

print(f"\n📋 Configuration Summary:")
print(config.get_workflow_summary())

print(f"\n🔄 To reuse this configuration:")
print(f"```python")
print(f"from omero_annotate_ai import load_config")
print(f"config = load_config('{config_filename}')")
print(f"```")

## 6. Cleanup

Close the OMERO connection when finished.

In [ ]:
# Close OMERO connection
if 'conn' in locals() and conn is not None:
    conn.close()
    print("🔌 OMERO connection closed")

print(f"\n🎉 Streamlined workflow completed!")
print(f"📊 Total images processed: {len(processed_images) if 'processed_images' in locals() else 0}")
print(f"💾 Configuration saved for future use")
print(f"✨ Ready for next annotation workflow!")

## Next Steps

### 🔄 Reusing Configurations
Save time by reusing your configuration:
```python
from omero_annotate_ai import load_config, create_pipeline
config = load_config('annotation_config_your_training_set.yaml')
pipeline = create_pipeline(config, conn)
```

### 🧪 Advanced Usage
- **Batch Processing**: Set `batch_size > 0` for large datasets
- **3D Processing**: Enable `three_d` for volumetric data
- **Patch Processing**: Use `use_patches` for large images
- **Read-only Mode**: Use `read_only_mode` for testing

### 📚 Documentation
- **Package Documentation**: See `CLAUDE.md` for development guidelines
- **API Reference**: Use `help(function_name)` for detailed documentation
- **Examples**: Check the `examples/` folder for more use cases

### 🛠️ Development
- **Testing**: Use `make test` to run the test suite
- **Linting**: Use `make lint` for code quality checks
- **Documentation**: Use `make docs` to build documentation